## denne filen er kun for notater!

In [2]:
# To display plots directly in the notebook:
%matplotlib inline

In [ ]:
#inputs man ofte trenger!
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib
import sklearn

In [7]:
#fetch gjør det mulig å henter datasett direkte fra nettstedet OpenML.org
from sklearn.datasets import fetch_openml

In [ ]:
#henter/ fatcher datasettet og lagrer det på data filen.
mnist = fetch_openml('mnist_784', version=1, data_home='./data', return_X_y=True)

In [ ]:
#gir oss tabell
mnist[0].head()

In [ ]:
#legger til elementer fra 0 -> 59 999 til treningsdata og fra 60 000 til testdata
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [ ]:
#X_treningsdata (hver rad er et bildet)
#.iloc[34500] = henter rad nummer 34 500 (altså ett spesifikt bilde)
#np.array(...) = konverterer raden til en NumPy-array
some_digit = np.array(X_train.iloc[34500])

In [ ]:
#etter koden over er some_digit en lang rekke med piskelverdier.
#vi vet at hvert bildet er på fomen 28*28, koden undrer endrer formatet fra en rekke til en 2D array.
some_digit_image = some_digit.reshape(28,28)

In [ ]:

fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(some_digit_image, cmap='gray')
ax.axis('off')
ax.set_title('Some Digit')
plt.show()

In [ ]:
#lager en funksjon som printer ut en litste av bilder, med et gitt antall, nb:
def plot_digits(to_plot, nb):
    """
    Plot a set of digit images.

    Parameters:
    to_plot (list): A list of image data to plot.
    nb (int): The number of images to plot.
    """
    f = plt.figure(figsize=(14, 14))
    for i in range(nb):
        plt.subplot(1, nb, i+1)
        plot_digit(to_plot[i])
    plt.show()

In [8]:
from sklearn.preprocessing import StandardScaler

In [ ]:
#når vi skal bruke forskjellige modeller som SGDClassifier er det viktig å normalisere dataene:
std = StandardScaler()
#her bergener vi gjennomsnittet og standardavviket på treningsdataene:
X_train_std = std.fit_transform(X_train)
#VIKTIG: her bruker vi de verdiene vi fant over til å normalisere test dataene. du skal ikke beregne nye gjennomsnitt og avvik!!!
X_test_std = std.transform(X_test)

In [ ]:
#trener SGDClassifier og bruker %%time for å finne ut hvor lang tid det tar.
%%time
sgd_clf.fit(X_train_std, y_train)

In [ ]:
#gjør det samme med random forest tree modellen.
%%time
rf_clf.fit(X_train, y_train)

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict

In [ ]:
# Warning: These computations take some time (depending on your CPU)...
sgd_train_pred = cross_val_predict(sgd_clf, X_train_std, y_train, n_jobs=-1)
rf_train_pred = cross_val_predict(rf_clf, X_train, y_train, n_jobs=-1)

In [ ]:
#her tester vi modellen basert på corss validation data. etter dette kan vi justere eller finpusse på modellen
#hvis dette trengs
accuracy_score(y_train, sgd_train_pred)

In [ ]:
#helt til slutt når vi tror vi har en vell fungerende modell, så kjører vi denne koden:
#PS: du kan kun kjøre denne 1 gang med samme test set!!! hvis du ikke er fornøyd med resultatet kan du enten lage modellen på nytt
#slik at modellen ikke har kjør testkoden eller du kan samle inn ny testdata hvis du har tilgang til dette. (ofte har vi ikke mer data)
accuracy_score(y_test, sgd_clf.predict(X_test))

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
#lager confusion matrix
sgd_cm = confusion_matrix(y_train, sgd_train_pred)
rf_cm = confusion_matrix(y_train, rf_train_pred)

In [ ]:
#kan gjøre dette for å sammenligne sgd og random forest:
print("SGD Confusion Matrix:")
print(sgd_cm)
print()
print("RF Confusion Matrix:")
print(rf_cm)

In [ ]:
#kan få en finere visuallisering med hjelp av ConfusionMatrixDisplay
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
# Create a confusion matrix display for two different classifiers
predictions = [sgd_train_pred, rf_train_pred]
classifier_names = ["SGDClassifier", "Random Forest"]

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
for i, (prediction, classifier_name) in enumerate(zip(predictions, classifier_names)):
    ConfusionMatrixDisplay.from_predictions(y_train, prediction, ax=axes[i])
    axes[i].set_title(classifier_name)
plt.show()

In [ ]:
#lager en metode for å plotte x som ble y og y som ble x:
# Code taken from the book _Géron: Hands-On Machine Learning with Scikit-Learn and TensorFlow_
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances.values]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = matplotlib.cm.binary, **options)
    plt.axis("off")

In [ ]:
def individual_errors(classifier, class_a, class_b):
    """
    Display individual errors made by a classifier.

    Parameters:
        classifier (object): The classifier to use.
        class_a (int): The first class to compare.
        class_b (int): The second class to compare.

    Returns:
        None
    """
    class_a, class_b = str(class_a), str(class_b)
    X_aa = X_train[(y_train == class_a) & (classifier.predict(X_train) == class_a)]  # Correct class a's
    X_ab = X_train[(y_train == class_a) & (classifier.predict(X_train) == class_b)]  # a's predicted as b's
    X_ba = X_train[(y_train == class_b) & (classifier.predict(X_train) == class_a)]  # b's predicted as a's
    X_bb = X_train[(y_train == class_b) & (classifier.predict(X_train) == class_b)]  # Correct b's

    plt.figure(figsize=(10, 10))
    plt.suptitle(f"Individual Errors: {class_a} vs {class_b}")

    if len(X_aa) > 0:
        ax1 = plt.subplot(221)
        plot_digits(X_aa[:25], images_per_row=5)
        ax1.text(0.5, 1.05, f"Correct {class_a}'s", ha='center', transform=ax1.transAxes)

    if len(X_ab) > 0:
        ax2 = plt.subplot(222)
        plot_digits(X_ab[:25], images_per_row=5)
        ax2.text(0.5, 1.05, f"{class_a}'s predicted as {class_b}'s", ha='center', transform=ax2.transAxes)

    if len(X_ba) > 0:
        ax3 = plt.subplot(223)
        plot_digits(X_ba[:25], images_per_row=5)
        ax3.text(0.5, 1.05, f"{class_b}'s predicted as {class_a}'s", ha='center', transform=ax3.transAxes)

    if len(X_bb) > 0:
        ax4 = plt.subplot(224)
        plot_digits(X_bb[:25], images_per_row=5)
        ax4.text(0.5, 1.05, f"Correct {class_b}'s", ha='center', transform=ax4.transAxes)
    plt.show()

In [ ]:
#eksempel bruk av koden: tester hvor mange 7-ere som ble riktig klassifisert og 7-ere klassifisert som 9-ere
# og antall 9-ere som ble riktig klassifisert og 9-ere som 
individual_errors(sgd_clf, 7, 9)